In [36]:
import pandas as pd
import numpy as np
import datetime as dt
from meteostat import Point, Daily

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_poisson_deviance
from sklearn.metrics import mean_gamma_deviance
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.metrics import brier_score_loss
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import jaccard_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import zero_one_loss
from sklearn.metrics import hinge_loss
from sklearn.metrics import zero_one_loss
from sklearn.metrics import log_loss
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import cohen_kappa_score


## data wrangling

In [29]:
df = pd.read_csv('C:\\Users\\shann\\Documents\\GitHub\\15095-project\\data\\international_matches_cleaned.csv')

In [30]:
df.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
0,9/2/2011,Argentina,Venezuela,South America,South America,9,44,1016,615,1,...,No,Win,77,64,80.8,88.3,85.2,72.2,73.7,75.5
1,9/2/2011,Belarus,Bosnia and Herzegovina,Europe,Europe,41,39,645,655,0,...,No,Lose,75,76,63.2,67.3,68.0,72.0,77.7,78.0
2,9/2/2011,Bulgaria,England,Europe,Europe,51,4,585,1176,0,...,No,Lose,80,83,70.8,73.7,73.5,84.0,81.3,85.8
3,9/2/2011,Ecuador,Jamaica,South America,North America,66,43,491,622,5,...,No,Win,69,63,70.5,74.7,76.8,65.2,72.3,71.8
4,9/2/2011,Germany,Austria,Europe,Europe,3,68,1329,472,6,...,No,Win,87,75,83.0,83.7,85.0,75.8,74.0,74.5


In [31]:
# variance of shoot_out column
# make shoot_out column binary
df['shoot_out'] = df['shoot_out'].apply(lambda x: 1 if x == 'yes' else 0)
df['shoot_out'].var() # no variance so we can drop this column

0.0

In [32]:
# remove variables that may cause leakage 
# remove variables that are not useful for modelling
df.drop(['home_team_score', # leakage
        'away_team_score', # leakage
        #'date', # not useful for modelling
        'shoot_out', # no variance
        'neutral_location', # not useful for this problem
        #'tournament'  # not useful for modelling
        ],
        inplace=True, axis=1)

In [34]:
# check tournament column counts
# df['tournament'].value_counts()

In [37]:
# use country and city to find weather data for each match
df[['city','country','tournament','date']].head(3)

,city,country,tournament,date
0,Calcutta,India,Friendly,9/2/2011
1,Minsk,Belarus,UEFA Euro qualification,9/2/2011
2,Sofia,Bulgaria,UEFA Euro qualification,9/2/2011


In [ ]:
df['date'] = pd.to_datetime(df['date'])